In [28]:
!pip install rdkit joblib tqdm pandas pyarrow gcsfs fastparquet mlflow google-cloud-storage google-cloud-aiplatform --quiet

error: uninstall-distutils-installed-package

× Cannot uninstall blinker 1.4
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
from google.colab import files
uploaded = files.upload()

In [4]:
# !gcloud auth activate-service-account --key-file=your-service-account-key.json

In [6]:
# !gcloud auth list

In [9]:
# !gcloud config set project kinetic-object-437418-v7

In [11]:
# !gcloud config list

In [13]:
# !gcloud auth login --no-launch-browser

In [15]:
# !gcloud auth list

In [17]:
# !gcloud storage buckets list

In [18]:
!pip install google-cloud-aiplatform --upgrade --quiet
# Install/Upgrade the specific 'google-cloud-aiplatform' package.
# The `--quiet` flag suppresses verbose output.
# The `--upgrade` flag

In [20]:
!pip install mlflow

In [21]:
from google.cloud import aiplatform

PROJECT_ID = "kinetic-object-437418-v7"
REGION = "us-east1"

# aiplatform.init(project=PROJECT_ID, location=REGION)


In [23]:
# import blinker
# print(blinker.__file__)  # Outputs the location of the installed package


In [24]:
# !rm -rf /usr/lib/python3/dist-packages/blinker/__init__.py


In [27]:
# !pip install --upgrade --force-reinstall --ignore-installed blinker



In [30]:
import mlflow
print(f"MLflow version: {mlflow.__version__}")

# Set MLflow to log runs locally
mlflow.set_tracking_uri("file:///content/mlruns")

# Verify the tracking URI
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

MLflow version: 2.18.0
Tracking URI: file:///content/mlruns


In [ ]:
# import mlflow

# # Set tracking URI (change to your MLflow server URL if applicable)
# MLFLOW_TRACKING_URI = "http://your-mlflow-server:5000"
# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [29]:
import pickle
import os
from google.cloud import storage

# Initialize the GCS client
client = storage.Client()

# Specify the GCS bucket and file paths
bucket_name = "drug_prediction"
file_name_X = "artifacts/X.pkl"
file_name_y = "artifacts/y.pkl"

# Local temporary directory to save downloaded files
local_directory = "/content/"
os.makedirs(local_directory, exist_ok=True)

# Function to download a file from GCS
def download_from_gcs(bucket_name, gcs_path, local_path):
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(gcs_path)
    blob.download_to_filename(local_path)
    print(f"Downloaded {gcs_path} to {local_path}")

# Download the pickle files from GCS
local_file_X = os.path.join(local_directory, "X.pkl")
local_file_y = os.path.join(local_directory, "y.pkl")
download_from_gcs(bucket_name, file_name_X, local_file_X)
download_from_gcs(bucket_name, file_name_y, local_file_y)

# Load the pickle files
with open(local_file_X, 'rb') as f:
    X = pickle.load(f)

with open(local_file_y, 'rb') as f:
    y = pickle.load(f)

# Print confirmation and details
print(f"Loaded X with shape: {len(X)} rows")
print(f"Loaded y with shape: {len(y)} rows")

Loaded X with shape: 10191006 rows
Loaded y with shape: 10191006 rows


In [31]:
!pip install xgboost catboost lightgbm --quiet

In [32]:
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    average_precision_score
)


In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

# Train and log model on Vertex AI
def train_and_log_model(model_name, model, X_train, X_val, y_train, y_val):
    # Train the model
    model = model.fit(X_train, y_train)

    # Predictions
    pred_model = model.predict(X_val)
    pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, "predict_proba") else None

    # Metrics
    accuracy = accuracy_score(y_val, pred_model)
    roc_auc = roc_auc_score(y_val, pred_proba) if pred_proba is not None else None
    weighted_map = (
        average_precision_score(y_val, pred_proba, average="weighted") if pred_proba is not None else None
    )

    # Classification report
    report = classification_report(y_val, pred_model)

    # Log metrics to Vertex AI
    experiment = aiplatform.Experiment(name="model_experiments")
    metrics = {
        "accuracy": accuracy,
    }
    if roc_auc is not None:
        metrics["roc_auc"] = roc_auc
    if weighted_map is not None:
        metrics["weighted_map"] = weighted_map

    experiment.run(
        run_name=model_name,
        run_params={
            "model_name": model_name,
        },
        run_metrics=metrics,
    )

    # Print metrics and classification report
    print(f"Classification Report for {model_name}:\n{report}")
    print(f"{model_name} - Accuracy: {accuracy:.4f}")
    if roc_auc is not None:
        print(f"ROC AUC: {roc_auc:.4f}")
    if weighted_map is not None:
        print(f"Weighted mAP: {weighted_map:.4f}")

# Main training workflow
def main():
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Calculate class imbalance ratio
    scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

    # Define models
    models = {
        "Logistic Regression": LogisticRegression(class_weight="balanced", max_iter=200),
        "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=30, class_weight="balanced", random_state=105, n_jobs=-1),
        "Decision Tree": DecisionTreeClassifier(max_depth=5, class_weight="balanced", random_state=42),
        "LightGBM": lgb.LGBMClassifier(n_estimators=50,
                                      max_depth=50,
                                      scale_pos_weight=scale_pos_weight,
                                      learning_rate=0.1),
        "XGBoost": xgb.XGBClassifier(n_estimators=100,
                                    max_depth=50,
                                    scale_pos_weight=scale_pos_weight,
                                    learning_rate=0.1,
                                    objective='binary:logistic',
                                    n_jobs=-1,
                                    random_state=42,
                                    tree_method='hist' ),
        "CatBoost": CatBoostClassifier(iterations=50,
                                      depth=6,
                                      learning_rate=0.1,
                                      verbose=0,
                                      scale_pos_weight=scale_pos_weight)
    }

    # Train and log each model
    for model_name, model in models.items():
        train_and_log_model(model_name, model, X_train, X_val, y_train, y_val)

if __name__ == "__main__":
    main()